# Inspect Warpy's Affine transformations
It seems it is just shifting and scaling in xy

In [1]:
from pathlib import Path
import json
import numpy as np

In [2]:
jsonfile = Path(r'transform.json')
with open(jsonfile) as f:
    data = json.load(f)
affine1 = np.array(data['realTransform_0']['affinetransform3d'])
tps = data['realTransform_1']['wrappedTransform']['wrappedTransform']
affine2 = np.array(data['realTransform_2']['affinetransform3d'])
srcPts = np.array(tps['srcPts'])
tgtPts = np.array(tps['tgtPts'])
affine1  # numbers in the 3D affine transform. Z is multiplied by 0.001 and then moved by -0.0005 ?

array([ 2.42534722e-04,  0.00000000e+00,  0.00000000e+00, -8.38757830e+00,
        0.00000000e+00,  2.42647059e-04,  0.00000000e+00, -7.34723162e+00,
        0.00000000e+00,  0.00000000e+00,  1.00000000e-03, -5.00000000e-04])

In [3]:
affine2  # numbers in the 3D affine transform. Z is multiplied by 1000 and then moved by 0.5 . That moves it back. Seems useless in 2D.

array([1.97186899e+03, 0.00000000e+00, 0.00000000e+00, 1.63200000e+04,
       0.00000000e+00, 1.97186899e+03, 0.00000000e+00, 1.41295000e+04,
       0.00000000e+00, 0.00000000e+00, 1.00000000e+03, 5.00000000e-01])

In [4]:
def reshape_affine(array_in):
    return np.array([[array_in[0], array_in[1],array_in[3]],
                     [array_in[4], array_in[5],array_in[7]],
                     [0.0, 0.0,1.0]])
affine1=reshape_affine(affine1)
affine2=reshape_affine(affine2)
print('Affine 1:')
print(affine1)
print('Affine 2:')
print(affine2)  
# typical representation of the affine transform for dot product transformation:

Affine 1:
[[ 2.42534722e-04  0.00000000e+00 -8.38757830e+00]
 [ 0.00000000e+00  2.42647059e-04 -7.34723162e+00]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Affine 2:
[[1.97186899e+03 0.00000000e+00 1.63200000e+04]
 [0.00000000e+00 1.97186899e+03 1.41295000e+04]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [5]:
affine_c = np.dot(np.transpose(affine1), np.transpose(affine2))
np.transpose(affine_c)  # combined affine transformation without the tps. 
# Pixels are scaled 0.478 and shifted in x and y.
# I suspect that large shifts and scales of the original two transforms are to map the 
# origine <0 ; 0> to the center of the image and normalize the range to be between -10 and 10 in x and y. 

array([[   0.4782467 ,    0.        , -219.20558867],
       [   0.        ,    0.47846821, -358.27822461],
       [   0.        ,    0.        ,    1.        ]])

In [6]:
srcPts = np.transpose(np.concatenate((srcPts, np.ones((1,len(srcPts[0]))))))
# bit silly to take the sourcepoints of the tps ofcourse, but just to show affine transformation any point would do.

In [7]:
resPts = np.dot(np.dot(srcPts, np.transpose(affine1)), np.transpose(affine2))

In [8]:
srcPts[0]  # original

array([-8.615536  , -7.00012132,  1.        ])

In [9]:
resPts[0]  # transformed without tps

array([-223.32594032, -361.62756014,    1.        ])

In [10]:
print(-8.615536*0.4782467 -219.20558867)
print(-7.00012132*0.47846821 -358.27822461)

-223.32594033073119
-361.6275601277632
